Plano de hoje
-------------

1. Ambiente de programação
    1. Interpretador interativo, Interpretador de arquivos, iPython
    1. Revisão da sintaxe de python
    1. Funções
    1. **Números "reais"**
    1. NumPy, SciPy: Matrizes e o mais
    1. MatPlotLib: gráficos

2. Usando o computador para calcular    
    1. **Indução e algoritmos recursivos: o método da bisseção**


# Números "reais"

É impossível representar todos os números reais no computador: este tem uma capacidade finita, enquanto aqueles são infinitos (e, pior, não-enumeráveis...). Assim, utilizaremos uma _aproximação_ dos mesmos.

A primeira observação é que, num computador, sempre existe um _maior número "real"_, pois um computador é capaz de representar apenas uma quantidade finita de números.
A segunda é que sempre existe um _próximo número "real"_, mais uma vez porque há apenas uma quantidade finita deles.

Dado isto, uma idéia simples para representar números reais seria considerar que a cada número inteiro corresponde um número real, através de uma fórmula do tipo
$$ real = \frac{inteiro}{10^9} $$
Isto é o que chamamos **ponto fixo**.
Este nome vem de imaginarmos que existe um ponto (implícito) no número inteiro que o computador vê,
e cuja posição é fixa (dada pelo $10^9$).

É claro que a escolha de $10^9$ para o denominador é completamente arbitrária,
e diferentes sistemas poderiam escolher diferentes denominadores.
Assim, uma representação em ponto fixo é dada por
$$\frac{inteiro}{den} \qquad \text{onde o denominador $den$ está fixo.}$$

### Exercício: Investigando as possibilidades do "ponto fixo"
Os números inteiros em um computador moderno são em geral os inteiros do intervalo $[- 2^{63}, 2^{63} - 1]$.

1. Deduza a precisão e a amplitude dos números que podem ser representados em ponto fixo se quisermos 15 casas decimais significativas, ou seja, respectivamente:
    - qual o **menor** "real" maior que zero;
    - qual o **maior** "real".
1. Idem para 5, 10 e 20 casas decimais.
1. Use a Wikipédia e descubra uma constante física que seja conhecida com o maior número possível de casas decimais.
1. Observe a precisão e a amplitude das constantes físicas que você procurou.
1. Conclua.

# O método da bisseção

O método da bisseção foi um dos primeiros métodos para achar raízes de funções "complicadas".
Ele é bastante simples, e esta simplicidade é também o seu poder.

Suponha que $f$ seja uma função contínua, e que $f(a)$ e $f(b)$ tenham sinais opostos.
O **Teorema do valor intermediário** garante que existe uma raiz de $f$ no segmento $[a,b]$.
Mais ainda, a _prova_ deste teorema é construtiva e nos dá um algoritmo para achar esta raiz:

- Considere $m = (a+b)/2$ o ponto médio de $[a,b]$, e calcule $f(m)$.
- Pode ser (com muita sorte) que $f(m) = 0$. Neste caso, acabou.
- Senão, $f(m)$ será positivo ou negativo, e como $f(a)$ e $f(b)$ têm sinais contrários, então
    - ou $f(a)$ e $f(m)$ têm sinais contrários
    - ou $f(b)$ e $f(m)$ têm sinais contrários
- Assim, podemos continuar buscando uma raíz na metade "certa" do intervalo original.

## Implementando a bisseção

Como vamos usar um computador, temos que levar em consideração que:

- As contas no computador podem não dar zero para uma raiz, por erro de precisão;
- E mesmo assim, em geral este método nunca termina, pois só obtemos a raiz pelo limite dos intervalos encaixados.

Portanto, e de forma bastante razoável, temos que limitar as nossas espectativas.
Uma forma interessante é pedir que a raiz seja calculada com uma certa precisão, digamos $10^{-6}$.
Neste caso, nós podemos **garantir** que esta precisão será atingida,
e portanto a resposta que o nosso método fornece não é apenas "um chute", mas ela é **certificada** pelo erro que a acompanha.

In [1]:
def bissecao(f,a,b,prec):
    """ Método da bisseção para uma função f no intervalo [a,b]. """
    m = (a+b)/2
    # Se já há precisão suficiente, retornamos o ponto médio do intervalo
    if abs(b - a) < prec: return m
    # Se f(m) == 0, achamos uma raiz exata!
    if f(m) == 0: return m

    # Senão, vamos por recorrência
    if f(m)*f(a) < 0: return bissecao(f,a,m,prec)
    else: return bissecao(f,m,b,prec)

## Testando

Uma das tarefas mais importantes ao se escrever um programa é ter certeza de que ele funciona corretamente.
Pode-se fazer isso de duas formas bastante complementares:

- Testando uma quantidade de casos interessantes e conhecidos.
- Provando (formalmente) que certas partes sempre irão dar o resultado correto.

A primeira maneira é mais simples, e já é capaz de nos dar uma certa confiança na qualidade do nosso programa.
A segunda requer uma formalização maior do problema original e da nossa linguagem de programação.
Vamos tentar desenvolver as duas em paralelo.

Um exemplo de teste: o cosseno tem uma raiz, $\pi/2$, no intervalo [0,3].

In [2]:
from math import cos
bissecao(cos,0,3, 1e-6)

1.5707963705062866

In [3]:
from math import pi
pi/2 - _

-4.371139006309477e-08

Nossa função `bissecao` é muito poderosa: ela aplica o método da bisseção a qualquer função que você possa programar em Python!
Por exemplo:

In [4]:
def estranha(x):
    return 2**x - x**2

In [5]:
bissecao(estranha, -1, 0, 1e-6)

-0.766664981842041

## Provando

Seria interessante provar que o nosso método de bisseção realmente funciona.
Para isso, precisamos especificar de forma mais precisa o que iremos calcular.

- Como já foi dito, queremos apenas que o valor retornado pela _procedure_ `bissecao` seja um número real $x$ cuja distância a alguma raiz $r$ de $f$ seja menor do que `prec`.
- Para isso precisamos também supor que $f$ seja contínua, e que $f(a)$ e $f(b)$ tenham sinais opostos.
- Além disso (e para simplificar), vamos supor que a _procedure_ `f` calcula **exatamente** o valor da função $f$.

É aqui que vamos usar a forma recursiva da nossa _procedure_ `bissecao` para guiar a demonstração.
Note que a única forma de sair da função é quando o teste sobre o valor absoluto é verdadeiro:
nos outros casos, sempre vamos "chamar de novo" a `bissecao`, e portanto podemos assumir (por indução!) que quando esta próxima chamada retornar, o resultado satisfaz as propriedades desejadas.

De novo, e com mais detalhes. Faça indução no número de vezes que a _procedure_ chama a si mesma:

- **[Base]** Se for zero, é porque ela retorna direto, porque o `if` é verdadeiro ; isso garante que a resposta está dentro de um intervalo cujo diâmetro é inferior a `prec`, e que contém a raiz $r$.
- **[Passo]** Se for $n+1$, a instância que será chamada "de dentro" só pode chamar a si mesma $n$ vezes ; por indução, ela retorna um real `x` a uma distância menor do que `prec` da raiz $r$, e este mesmo valor será retornado pela instância principal.

# Uma digressão sobre nomenclatura

Vamos, ao longo do curso, tratar de funções de dois tipos:
- As funções matemáticas, como seno, exponencial, polinômios, etc.
- As funções do seu programa em Python.

O simples fato de os nomes serem os mesmos já pode levar a confusão.
Mas, principalmente, a maior parte das funções-programas que nós veremos durante o curso irão trabalhar com funções-contas.
Assim, à imagem do método da bisseção, teremos muitas vezes funções da forma

``` def metodoX(f, a, b, c, ...):
    contas...
    contas...
    return
```

que irá aplicar o método X à função `f`, uma função-programa que _calcula_ a função matemática muitas vezes também chamada $f$.

Para tentar diminuir a ambigüidade, sempre que necessário iremos usar uma palavra computacional, tal como

- procedimento, método, rotina, algoritmo

para indicar que se trata de um programa em Python, e iremos reservar a palavra função (às vezes incrementada do adjetivo "matemática") para as funções abstratas da Matemática.

### Exercício: bisseção, precisão, e ponto fixo

Imagine o que acontece quando sempre escolhemos o intervalo da esquerda ao aplicar o método da bisseção.
O último intervalo que pode ser produzido desta forma é $[a, next(a)]$, onde $next(a)$ é o "próximo número real" para o seu computador.

Use esta observação para escrever uma _procedure_ que calcula o "próximo número real".

In [6]:
def biss_esquerda(a,b):
    """ Efetua a divisão do intervalo [a,b], recursivamente, sempre pegando o intervalo da esquerda.
        Quando o intervalo contém apenas dois números, retorna o maior.
    """
    pass

def next_real(x):
    """ Retorna o próximo número real."""
    pass

Façamos alguns testes:

In [7]:
next_real(1.0), next_real(next_real(1.0)), next_real(10.0), next_real(1000.0)

(1.0000000000000002,
 1.0000000000000004,
 10.000000000000002,
 1000.0000000000001)

O seu computador usa pontos fixos?

Para confirmar, escreva uma rotina que dê a diferença absoluta e relativa entre um número `x` e o seu sucessor.

In [8]:
def diff_next(x):
    pass

In [9]:
diff_next(1.), diff_next(10.), diff_next(1000.)

((2.220446049250313e-16, 2.220446049250313e-16),
 (1.7763568394002505e-15, 1.7763568394002506e-16),
 (1.1368683772161603e-13, 1.1368683772161603e-16))

Antes de continuar testando, vamos tentar provar que o nosso programa está certo.

- A hipótese básica sobre a aritmética do computador é que ele sempre retorna "o número representável mais próximo do valor real da conta". Ou seja, se tivermos números $a$ e $b$ e pedirmos para o computador calcular `c = a+b`, ele retornará o número que "ele conhece" que está mais próximo de $a+b$.
- No caso de "empate", ou seja, haver dois números `x` e `y = next_real(x)` que são equidistantes do resultado $a+b$, o computador pode retornar `x` ou `y`.

Apenas com isso, não podemos ainda garantir que o nosso programa está correto. Quais outras hipóteses você precisa adicionar para garantir o bom funcionamento do seu algoritmo?

Algumas ideias:

- Será que precisamos de uma hipótese sobre o espaçamento entre dois números consecutivos?
- Será que precisamos de alguma hipótese sobre o zero?
- Será que precisamos que o computador conheça `-x` se ele conhece `x`?
- ...

Agora que nós estudamos o nosso algoritmo, e sabemos algumas condições que o farão dar o resultado certo, vamos submetê-lo a mais alguns testes.

Começamos com os números negativos.

In [10]:
next_real(-1.0), next_real(next_real(-1.0)), next_real(-10.0), next_real(-1000.0)

(-0.9999999999999999,
 -0.9999999999999998,
 -9.999999999999998,
 -999.9999999999999)

E alguns números BEM pequenos!

In [11]:
next_real(0), next_real(next_real(0)), next_real(1e-100), next_real(-1e-100)

(5e-324, 1e-323, 1.0000000000000001e-100, -9.999999999999999e-101)

Como seria a função para o número "real" _anterior_? Será que alguma hipótese sobre os números negativos pode tornar esta função bem mais fácil?